<a href="https://colab.research.google.com/github/SilviaCalzon/IA-School/blob/main/Copie_de_Projet_serie_temporelle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)
library(forecast)

#ouverture du fichier
fichier = "coronavirus_ts.csv"
mydata <- read_csv2(fichier)
View(mydata)

#transformation des dates en facteur
mydata$date <- as.factor(mydata$date)

#affichage des descripteurs de notre table de données
head(mydata, 10)
summary(mydata)

#enlever les lignes avec valeurs manquantes
mydata = na.omit(mydata)

#Visualisation
ggplot(mydata) +
  aes(x = date, colour = guerisons, weight = cas) +
  geom_bar(fill = "#B22222") +
  scale_color_distiller(palette = "RdGy", direction = -1) +
  labs(
    x = "Temps",
    y = "Nombre de cas",
    title = "Evolution du nombre de cas et des guérisons en fonction du temps",
    color = "Nombre de guérisons"
  ) +
  ggthemes::theme_solarized_2()

#Outliers
library(VIM)
VIM::matrixplot(mydata)

#Imputation multiples pour prévenir les erreurs liés à l'imputation ou la complétion de données
library(mice)
mydatamice <- mice(mydata, method = "rf")
mydataclean <- complete(mydatamice)

#Visualisation de la distribution des données dates
boxplot(mydataclean$date)
str(mydataclean)
tail(mydataclean$date)

#Visualisation de la distribution du nombre de guérison
guerison = mydataclean %>% group_by(date) %>% summarise(mean(guerisons))
head(guerison)
tail(guerison)
plot(guerison)
ggplot(mydataclean) +
  aes(x = date, y = guerisons) +
  geom_line(size = 0.5, colour = "#8F622B") +
  labs(x = "Temps", 
       y = "Nombre de guérison", title = "Evolution du nombre de guérison dans le temps") +
  ggthemes::theme_solarized_2()

#Série temporelle des guérisons
serie_guerison = ts(guerison$'mean(guerisons)', frequency = 7)
serie_guerison
plot(serie_guerison, lwd=3, col = "red")

autoplot(serie_guerison)

acf_g<-acf(serie_guerison)
print(data.frame(acf_g$lag,acf_g$acf)[1:10,])

result_pacf=pacf(serie_guerison)
result_pacf

meanf(serie_guerison,31)
naive(serie_guerison,31)
snaive(serieguerison,31)
rwf(serie_guerison,31, drift=TRUE)

autoplot(serie_guerison) +
  autolayer(meanf(serie_guerison, h=31), series="Moyenne", PI=FALSE) +
  autolayer(naive(serie_guerison, h=31), series="Naïve", PI=FALSE) +
  autolayer(snaive(serie_guerison, h=31), series="Naïve saisonnière", PI=FALSE) +
  autolayer(rwf(serie_guerison,31, drift=TRUE), series="Naïve dérive", PI=FALSE)+
  ggtitle("Prevision du nombre de guérison pour les deux prochaines années") +
  xlab("Trimestre") + 
  ylab("Nombre de guérison") +
  guides(colour=guide_legend(title="Prevision")) 

#Prédiction ets et arima et tbats des guérisons sur sa série temporelle
f_ets = forecast(ets(serie_guerison), h=150)  
plot(f_ets,col="red",lwd=3)
f_aa = forecast(auto.arima(serie_guerison), h=107)
plot(f_aa,col="green",lwd=3)
f_tbats = forecast(tbats(serie_guerison), h=107)
plot(f_tbats,col="green",lwd=3)


#Décomposition des guérisons (valeurs observées, tendance, périodicité, hasard)
decomposition=decompose(serie_guerison)
decomposition
plot(decomposition,lwd=3)

ggseasonplot(serie_guerison, polar=TRUE) +
  ylab("Nombre de cas de guérison") +
  ggtitle("Seasonal plot: nombre de cas de guérisons par jour") 
Box.test(serie_guerison, lag = 4, type = "Ljung-Box")

#Retirer donner monde
tab <- subset(mydataclean, nom != 'Monde')
world_map <- map_data("world")

#Carte du nombre de cas dans le monde
library(rworldmap)

tab <-  mydata %>% group_by(nom) %>% summarise(guerisons)
joinData <- joinCountryData2Map(tab,
                                joinCode = "NAME",
                                nameJoinColumn = "nom")
theMap <- mapCountryData(joinData, nameColumnToPlot="guerisons", addLegend=FALSE)
do.call(addMapLegend, c(theMap, legendWidth=1, legendMar = 2))


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



ERROR: ignored